# Library setting

In [1]:
import sys
sys.path.append('/Volumes/KHJ/Github/hyuckjinkim/lib-python')

from base import gc_collect_all, setdiff
from filesystem_utils import mkdir
from environment import LocalFontSetting, ColabInstallFont, ColabFontSetting
from graph import abline, actual_prediction_scatterplot
from data_prepare import (
    get_holiday, reduce_mem_usage, delete_unique_columns,
    TypeController, CategoricalQuantileCalculator,
    GroupScaler, OneHotEncoder, InteractionTerm, TargetTransform,
)

In [2]:
gc_collect_all()
LocalFontSetting()

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from tqdm import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd
pd.reset_option('display')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', None)
pd.set_option('mode.chained_assignment', None)

import datetime
import glob

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
def return_unique_columns(data):
    unique_info = data.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    return unique_cols

In [5]:
class CFG:
    SEED = 42
    SUBSET_DEPTH = 3
    TARGET = 'ECLO'
    TARGET_TRANSFORMATION = 'identity'

<br></br>

# Data

## Data load

In [6]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')

# cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv',encoding='cp949')
# security_light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv',encoding='cp949',low_memory=False)
# child_area_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv',encoding='cp949')
# parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv',encoding='cp949')

<br>

## Preprocessing

In [7]:
def preprocessing(data):
    d = data.copy()
    
    # (1) test data에 없는 컬럼 제거
    no_columns_in_test = ['사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
                          '가해운전자 상해정도','피해운전자 차종','피해운전자 성별','피해운전자 연령','피해운전자 상해정도']
    no_columns_in_test = list(set(d.columns)&set(no_columns_in_test))
    d.drop(no_columns_in_test,axis=1,inplace=True)
    
    # (1) 시군구: 도시 / 구 / 동
    location_pattern = r'(\S+) (\S+) (\S+)'
    d[['도시','구','동']] = d['시군구'].str.extract(location_pattern)
    d.drop(['시군구','도시'],axis=1,inplace=True)
    
    # (2) 도로형태
    d[['도로구분','도로형태상세']] = np.stack(d['도로형태'].str.split(' - '))
    d['주차장여부'] = d['도로구분'].map({'단일로':0,'교차로':0,'기타':np.nan,'주차장':1,'미분류':np.nan})
    d['도로여부']    = d['도로구분'].map({'단일로':1,'교차로':1,'기타':np.nan,'주차장':np.nan,'미분류':np.nan})
    d.drop('도로형태',axis=1,inplace=True)
    for col in ['주차장여부','도로여부']:
        d[col] = d[col].fillna(0)
        
    # (3) 기상상태
    d['기상상태맑음여부'] = np.where(d['기상상태']=='맑음',1,0)
    d['노면상태건조여부'] = np.where(d['노면상태']=='건조',1,0)

    return d

def derived_features(data):
    d = data.copy()
    date = d['사고일시'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y-%m-%d %H'))
    
    # (1) date columns
    d['year']       = date.dt.year
    d['month']      = date.dt.month
    d['day']        = date.dt.day
    d['hour']       = date.dt.hour
    d['dayofweek']  = date.dt.dayofweek
    d['weekend']    = date.dt.dayofweek.isin([5,6]).astype(int)
    d['week']       = [t.isocalendar()[1] for t in date]
    d['season']     = d['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})
    
    # 저어어어어엉님 코드 (https://dacon.io/competitions/official/236176/codeshare/9381?page=1&dtype=recent)
    # 주차 누적값
    week_list=[]
    for i in range(len(d)) :
        if d['year'][i] == 2019 :
            week_list.append(int(d['week'][i]))
        elif d['year'][i] == 2020 :
            week_list.append(int(d['week'][i])+52)
        elif d['year'][i] == 2021 :
            week_list.append(int(d['week'][i])+52+53)
        elif d['year'][i] == 2022 :
            week_list.append(int(d['week'][i])+52+53+53)
        else:
            raise ValueError('Unknown year value')
    d['week_num']= week_list
    
    # datetime 패키지에서 19년 12월 마지막주가 첫째주로 들어가는거 발견하여 수정
    d.loc[(d['year']==2019) & (d['month']==12) & (d['day']==30), 'week_num'] = 52
    d.loc[(d['year']==2019) & (d['month']==12) & (d['day']==31), 'week_num'] = 52

    # (2) is holiday & is dayoff
    holiday_list = get_holiday(d['year'].unique())
    d['is_holiday'] = date.isin(holiday_list).astype(int)
    d['is_dayoff']  = ((d.is_holiday==1) | (d.weekend==1)).astype(int)
    
    # (3) unuse features
    unuse_features = ['ID','사고일시','요일','사망자수','중상자수','경상자수','부상자수']
    unuse_features = list(set(d.columns)&set(unuse_features))
    d.drop(columns=unuse_features,inplace=True)
    
    num_to_cat = ['year','month','day','hour','dayofweek','week','season']
    d[num_to_cat] = d[num_to_cat].astype(str)
    
    return d

In [8]:
train_df = preprocessing(train_df)
train_df = derived_features(train_df)

test_df = preprocessing(test_df)
test_df = derived_features(test_df)

In [9]:
train_df.head()

,기상상태,노면상태,사고유형,ECLO,구,동,도로구분,도로형태상세,주차장여부,도로여부,기상상태맑음여부,노면상태건조여부,year,month,day,hour,dayofweek,weekend,week,season,week_num,is_holiday,is_dayoff
0,맑음,건조,차대사람,5,중구,대신동,단일로,기타,0.0,1.0,1,1,2019,1,1,0,1,0,1,0,1,1,1
1,흐림,건조,차대사람,3,달서구,감삼동,단일로,기타,0.0,1.0,0,1,2019,1,1,0,1,0,1,0,1,1,1
2,맑음,건조,차대사람,3,수성구,두산동,단일로,기타,0.0,1.0,1,1,2019,1,1,1,1,0,1,0,1,0,0
3,맑음,건조,차대차,5,북구,복현동,단일로,기타,0.0,1.0,1,1,2019,1,1,2,1,0,1,0,1,0,0
4,맑음,건조,차대차,3,동구,신암동,단일로,기타,0.0,1.0,1,1,2019,1,1,4,1,0,1,0,1,0,0


<br>

## Merge addtional information

In [10]:
def frequency_merge(data,info_data,group,target,fillna_method):
    assert fillna_method in ['zero','min','max','avg'], \
        "fillna_method must be one of ['zero','min','max','avg']"
    prefix = f'{target}_'
    
    d = data.copy()
    info_d = info_data.copy()
    
    freq_data = info_data\
        .groupby(group)[target]\
        .apply(lambda x: x.value_counts())\
        .reset_index()\
        .pivot_table(index=group,columns=f'level_{len(group)}',values=target)\
        .add_prefix(prefix)\
        .reset_index()
    d = pd.merge(d,freq_data,how='left',on=group)
    
    cols = info_data[target].dropna().unique()
    cols = [f'{target}_{col}' for col in cols]
    cols = list(set(cols)&set(d.columns))
    for col in cols:
        if fillna_method=='zero':
            fillna_value = 0
        elif fillna_method=='min':
            fillna_value = d[col].min()
        elif fillna_method=='max':
            fillna_value = d[col].max()
        elif fillna_method=='avg':
            fillna_value = d[col].mean()
        d[col].fillna(fillna_value,inplace=True)
        
    assert len(data)==len(d), \
        "duplicated"
        
    return d

In [11]:
def agg_merge(data,info_data,group,target,agg,fillna_method=None):
    assert fillna_method in [None,'zero','min','max','avg'], \
        "fillna_method must be one of ['zero','min','max','avg']"
    assert agg in ['min','max','avg','sum'], \
        "agg must be one of ['min','max','avg','sum']"
    
    if agg=='min':
        aggfunc = np.nanmin
    elif agg=='max':
        aggfunc = np.nanmax
    elif agg=='avg':
        aggfunc = np.nanmean
    elif agg=='sum':
        aggfunc = np.nansum
        
    d = data.copy()
    info_d = info_data.copy()
        
    agg_df = info_d\
        .groupby(group)[target]\
        .apply(lambda x: aggfunc(x))\
        .reset_index()\
        .rename(columns={target:f'{target}_{agg}'})
    d = pd.merge(d,agg_df,how='left',on=group)
    
    if fillna_method is not None:
        if fillna_method=='zero':
            fillna_value = 0
        elif fillna_method=='min':
            fillna_value = d[f'{target}_{agg}'].min()
        elif fillna_method=='max':
            fillna_value = d[f'{target}_{agg}'].max()
        elif fillna_method=='avg':
            fillna_value = d[f'{target}_{agg}'].mean()
        d[f'{target}_{agg}'].fillna(fillna_value,inplace=True)
    
    assert len(data)==len(d), \
        "duplicated"
    
    return d

<br>

### CCTV information

In [12]:
def add_cctv_info(data):
    d = data.copy()
    cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv',encoding='cp949')

    # 소재지지번주소 -> 도시 / 구 / 동 / 지번
    location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
    cctv_df[['도시','구','동','지번']] = cctv_df['소재지지번주소'].str.extract(location_pattern)
    cctv_df.drop(['소재지지번주소','도시','지번'],axis=1,inplace=True)
    
    # 단속구분별 cctv 수
    tmp = cctv_df[cctv_df['단속구분']!=99]
    d = frequency_merge(d,tmp,group=['구','동'],target='단속구분',fillna_method='zero')

    # 도로노선방향별 cctv 수
    d = frequency_merge(d,cctv_df,group=['구','동'],target='도로노선방향',fillna_method='zero')
    d.drop('도로노선방향_3',axis=1,inplace=True) # 단속구분_4랑 corr 100%

    # 제한속도 평균
    tmp = cctv_df[cctv_df['제한속도']!=0]
    d = agg_merge(d,tmp,group=['구','동'],target='제한속도',agg='avg',fillna_method='max')

    # 설치연도 평균
    d = agg_merge(d,cctv_df,group=['구','동'],target='설치연도',agg='avg',fillna_method='min')

    # 보호구역구분: 1,2인 경우 보호구역으로 보고, 나머지(99,nan)는 보호구역이 아닌 것으로 봄
    cctv_df['보호구역여부'] = np.where(cctv_df['보호구역구분'].isin([1,2]),1,0)
    d = agg_merge(d,cctv_df,group=['구','동'],target='보호구역여부',agg='sum',fillna_method='zero')
    
    # 위도,경도
    d = agg_merge(d,cctv_df,group=['구','동'],target='위도',agg='avg',fillna_method=None)
    d = agg_merge(d,cctv_df,group=['구','동'],target='경도',agg='avg',fillna_method=None)
    
    use_cols = setdiff(d.columns,data.columns)
    rename_dict = {col:'CCTV_'+col for col in use_cols}
    d = d.rename(columns=rename_dict)
    
    assert len(data)==len(d), \
        "duplicated"
    
    return d

In [13]:
train_df = add_cctv_info(train_df)
test_df  = add_cctv_info(test_df)

<br>

### Light information

In [14]:
def add_light_info(data):
    d = data.copy()
    light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv',encoding='cp949',low_memory=False)
    
    # 소재지지번주소 -> 도시 / 구 / 동 / 지번
    location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
    light_df[['도시','구','동','지번']] = light_df['소재지지번주소'].str.extract(location_pattern)
    light_df.drop(['소재지지번주소','도시','지번'],axis=1,inplace=True)
    
    # 보안등 설치수
    d = agg_merge(d,light_df,group=['구','동'],target='설치개수',agg='sum',fillna_method='zero')
    
    # 보안등 설치연도
    d = agg_merge(d,light_df,group=['구','동'],target='설치연도',agg='avg',fillna_method='min')
    
    # 보안등 설치형태의 빈도
    d = frequency_merge(d,light_df,group=['구','동'],target='설치형태',fillna_method='zero')
    
    # 위도,경도
    d = agg_merge(d,light_df,group=['구','동'],target='위도',agg='avg',fillna_method=None)
    d = agg_merge(d,light_df,group=['구','동'],target='경도',agg='avg',fillna_method=None)
    
    use_cols = setdiff(d.columns,data.columns)
    rename_dict = {col:'보안등_'+col for col in use_cols}
    d = d.rename(columns=rename_dict)
    
    assert len(data)==len(d), \
        "duplicated"
    
    return d

In [15]:
train_df = add_light_info(train_df)
test_df  = add_light_info(test_df)

/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\


<br>

### Child area information

In [16]:
def add_child_area_info(data):
    d = data.copy()
    child_area_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv',encoding='cp949')

    # 소재지지번주소 -> 도시 / 구 / 동 / 지번
    location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
    child_area_df[['도시','구','동','지번']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
    child_area_df.drop(['소재지지번주소','도시','지번'],axis=1,inplace=True)

    # 시설종류의 빈도
    d = frequency_merge(d,child_area_df,group=['구','동'],target='시설종류',fillna_method='zero')

    # 관할경찰서명의 빈도
    d = frequency_merge(d,child_area_df,group=['구','동'],target='관할경찰서명',fillna_method='zero')

    # CCTV설치여부의 빈도
    d = frequency_merge(d,child_area_df,group=['구','동'],target='CCTV설치여부',fillna_method='zero')

    # CCTV설치대수
    d = agg_merge(d,child_area_df,group=['구','동'],target='CCTV설치대수',agg='sum',fillna_method='zero')

    # 보호구역도로폭
    child_area_df['보호구역도로폭'] = child_area_df['보호구역도로폭'].apply(lambda x: sum([float(s) for s in str(x).split('~')]))
    d = agg_merge(d,child_area_df,group=['구','동'],target='보호구역도로폭',agg='avg',fillna_method='min')
    
    # 위도,경도
    d = agg_merge(d,child_area_df,group=['구','동'],target='위도',agg='avg',fillna_method=None)
    d = agg_merge(d,child_area_df,group=['구','동'],target='경도',agg='avg',fillna_method=None)
    
    use_cols = setdiff(d.columns,data.columns)
    rename_dict = {col:'어린이보호구역_'+col for col in use_cols}
    d = d.rename(columns=rename_dict)
    
    assert len(data)==len(d), \
        "duplicated"

    return d

In [17]:
train_df = add_child_area_info(train_df)
test_df  = add_child_area_info(test_df)

/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\


<br>

### Parking information

In [18]:
def add_parking_info(data):
    d = data.copy()
    parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv',encoding='cp949')

    # 소재지지번주소 -> 도시 / 구 / 동 / 지번
    location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
    parking_df[['도시','구','동','지번']] = parking_df['소재지지번주소'].str.extract(location_pattern)
    parking_df.drop(['소재지지번주소','도시','지번'],axis=1,inplace=True)

    # 주차장구분의 빈도
    d = frequency_merge(d,parking_df,group=['구','동'],target='주차장구분',fillna_method='zero')

    # 주차장유형의 빈도
    d = frequency_merge(d,parking_df,group=['구','동'],target='주차장유형',fillna_method='zero')

    # 급지구분의 빈도
    d = frequency_merge(d,parking_df,group=['구','동'],target='급지구분',fillna_method='zero')

    # 요금정보의 빈도
    d = frequency_merge(d,parking_df,group=['구','동'],target='요금정보',fillna_method='zero')

    # 주차구획수
    d = agg_merge(d,parking_df,group=['구','동'],target='주차구획수',agg='avg',fillna_method='zero')

    # 월정기권요금 (0제거)
    parking_df['월정기권요금'] = parking_df['월정기권요금'].replace(0,np.nan)
    d = agg_merge(d,parking_df,group=['구','동'],target='월정기권요금',agg='avg',fillna_method='zero')

    # 평일/토요일/공휴일 운영시간
    for daytype in ['평일','토요일','공휴일']:
        new_col = f'{daytype}운영시간'
        starttime_col = f'{daytype}운영시작시각'
        endtime_col = f'{daytype}운영종료시각'

        parking_df[new_col] = [round((datetime.datetime.strptime(b,'%H:%M')-datetime.datetime.strptime(a,'%H:%M')).seconds/3600,0)
                               for a,b in parking_df[[starttime_col,endtime_col]].values]
        parking_df[new_col] = [24 if t==0 else t for t in parking_df[new_col]]
        parking_df[new_col] = [t if w.find(daytype)>=0 else 0 for t,w in parking_df[[new_col,'운영요일']].values]

        d = agg_merge(d,parking_df,group=['구','동'],target=new_col,agg='avg',fillna_method='zero')
        
    use_cols = setdiff(d.columns,data.columns)
    rename_dict = {col:'주차장_'+col for col in use_cols}
    d = d.rename(columns=rename_dict)
        
    assert len(data)==len(d), \
        "duplicated"

    return d

In [19]:
train_df = add_parking_info(train_df)
test_df  = add_parking_info(test_df)

/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\
/var/folders/4s/0m_928_d0z7c8r5nngpf8rtm0000gn/T/ipykernel_40966/3856018639.py:21: RuntimeWarning: Mean of empty slice
  .apply(lambda x: aggfunc(x))\


<br>

## Feature engineering

### fill N/A values by averaging

In [20]:
def coordinates_fillna(values):
    x = np.array(values).flatten()
    if sum(np.isnan(x)) == len(x):
        return [np.nan]*len(x)
    else:
        fillna_value = np.mean(x[~np.isnan(x)])
        return [fillna_value if np.isnan(v) else v for v in x]

In [21]:
latitude_cols  = train_df.columns[train_df.columns.str.contains('위도')].tolist()
longitude_cols = train_df.columns[train_df.columns.str.contains('경도')].tolist()

train_df[latitude_cols] = np.stack(train_df[latitude_cols].apply(coordinates_fillna,axis=1))
test_df [latitude_cols] = np.stack(test_df [latitude_cols].apply(coordinates_fillna,axis=1))

train_df[longitude_cols] = np.stack(train_df[longitude_cols].apply(coordinates_fillna,axis=1))
test_df [longitude_cols] = np.stack(test_df [longitude_cols].apply(coordinates_fillna,axis=1))

<br>

### Get missing latitude and longitude via geocode

In [22]:
# (참조) https://medium.com/@hazallgultekin/convert-address-to-latitude-longitude-using-python-21844da3d032
from geopy.geocoders import Nominatim
def get_geocode(address):
    # calling the Nominatim tool and create Nominatim class
    loc = Nominatim(user_agent="Geopy Library")

    # entering the location name
    getLoc = loc.geocode(address)

    return getLoc

In [23]:
col = latitude_cols[0]
pd.crosstab(
    np.where(train_df[col].isnull(),1,0),
    np.where(train_df[col].isnull(),1,0),
)

col_0,0,1
row_0,,
0,39286,0
1,0,323


In [24]:
# 위도/경도가 Null인 구/동을 모두 모음
gu_dong_data = pd.concat([
    train_df[['구','동']][train_df[col].isnull()],
    test_df [['구','동']][test_df [col].isnull()],
],axis=0).drop_duplicates().values

# 위의 구/동으로부터 위도/경도를 search (7개)
coord_info = []
for gu,dong in tqdm(gu_dong_data):
    address = f'대구광역시 {gu} {dong}'
    info = get_geocode(address)
    if info is not None:
        coord_info.append([gu,dong,info.latitude,info.longitude])
    else:
        coord_info.append([gu,dong,None,None])
coord_info = pd.DataFrame(coord_info,columns=['구','동','위도','경도'])
display(coord_info.isnull().sum())

100%|█████████████████████████████████████████████| 7/7 [00:06<00:00,  1.05it/s]


구     0
동     0
위도    0
경도    0
dtype: int64

In [25]:
def coordinates_fillna_with_geocode_values(values):
    x = np.array(values).flatten()
    if np.isnan(x[-1]):
        return x[:-1]
    else:
        return [x[-1]]*(len(x)-1)

In [26]:
train_df = pd.merge(train_df,coord_info,how='left',on=['구','동'])
train_df[latitude_cols]  = np.stack(train_df[latitude_cols +['위도']].apply(coordinates_fillna_with_geocode_values,axis=1))
train_df[longitude_cols] = np.stack(train_df[longitude_cols+['위도']].apply(coordinates_fillna_with_geocode_values,axis=1))
train_df.drop(['위도','경도'],axis=1,inplace=True)

test_df = pd.merge(test_df,coord_info,how='left',on=['구','동'])
test_df[latitude_cols]  = np.stack(test_df[latitude_cols +['위도']].apply(coordinates_fillna_with_geocode_values,axis=1))
test_df[longitude_cols] = np.stack(test_df[longitude_cols+['위도']].apply(coordinates_fillna_with_geocode_values,axis=1))
test_df .drop(['위도','경도'],axis=1,inplace=True)

In [27]:
train_df.isnull().sum().sum(),test_df.isnull().sum().sum()

(0, 0)

In [28]:
train_df.shape

(39609, 73)

<br>

## Target transformation

In [29]:
target_transform = TargetTransform(func=CFG.TARGET_TRANSFORMATION, offset=1)
train_df[CFG.TARGET] = target_transform.fit_transform(train_df[CFG.TARGET])

In [30]:
train_df.head(1)

,기상상태,노면상태,사고유형,ECLO,구,동,도로구분,도로형태상세,주차장여부,도로여부,기상상태맑음여부,노면상태건조여부,year,month,day,hour,dayofweek,weekend,week,season,week_num,is_holiday,is_dayoff,CCTV_단속구분_1,CCTV_단속구분_2,CCTV_단속구분_4,CCTV_도로노선방향_1,CCTV_도로노선방향_2,CCTV_제한속도_avg,CCTV_설치연도_avg,CCTV_보호구역여부_sum,CCTV_위도_avg,CCTV_경도_avg,보안등_설치개수_sum,보안등_설치연도_avg,보안등_설치형태_건축물,보안등_설치형태_전용주,보안등_설치형태_한전주,보안등_위도_avg,보안등_경도_avg,어린이보호구역_시설종류_어린이집,어린이보호구역_시설종류_유치원,어린이보호구역_시설종류_초등학교,어린이보호구역_시설종류_특수학교,어린이보호구역_시설종류_학원,어린이보호구역_관할경찰서명_강북경찰서,어린이보호구역_관할경찰서명_남부경찰서,어린이보호구역_관할경찰서명_달성경찰서,어린이보호구역_관할경찰서명_대구광역시 중부경찰서,어린이보호구역_관할경찰서명_북부경찰서,어린이보호구역_관할경찰서명_서부경찰서,어린이보호구역_관할경찰서명_수성경찰서,어린이보호구역_CCTV설치여부_N,어린이보호구역_CCTV설치여부_Y,어린이보호구역_CCTV설치대수_sum,어린이보호구역_보호구역도로폭_avg,어린이보호구역_위도_avg,어린이보호구역_경도_avg,주차장_주차장구분_공영,주차장_주차장구분_민영,주차장_주차장유형_노상,주차장_주차장유형_노외,주차장_급지구분_1,주차장_급지구분_2,주차장_급지구분_3,주차장_요금정보_무료,주차장_요금정보_유료,주차장_요금정보_혼합,주차장_주차구획수_avg,주차장_월정기권요금_avg,주차장_평일운영시간_avg,주차장_토요일운영시간_avg,주차장_공휴일운영시간_avg
0,맑음,건조,차대사람,5.0,중구,대신동,단일로,기타,0.0,1.0,1,1,2019,1,1,0,1,0,1,0,1,1,1,0.0,1.0,4.0,0.0,1.0,40.0,2013.0,2.0,35.868237,128.580886,391.0,2007.076046,177.0,30.0,135.0,35.867981,128.579156,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,13.0,26.0,35.868541,128.581033,0.0,11.0,0.0,11.0,11.0,0.0,0.0,0.0,11.0,0.0,45.454545,0.0,12.636364,12.636364,2.181818


<br>

## Quantile values of target

In [31]:
cat_features = train_df.columns[train_df.dtypes=='object'].tolist()
dummy_features = ['주차장여부','도로여부','기상상태맑음여부','노면상태건조여부','weekend','is_holiday','is_dayoff']
num_features = setdiff(train_df.columns,cat_features+dummy_features+[CFG.TARGET])

In [32]:
# # feature engineering
# calculator = CategoricalQuantileCalculator()
# calculator.fit(
#     data=train_df,
#     test_data=test_df,
#     target_feature=CFG.TARGET,
#     cat_features=cat_features,
#     subset_depth=CFG.SUBSET_DEPTH,
# )
# train_df = calculator.transform(train_df)
# test_df  = calculator.transform(test_df)

In [33]:
train_df.shape

(39609, 73)

<br>

## TargetEncoder

In [34]:
# target값이 높은 category에 높은 숫자를 부여
from category_encoders.target_encoder import TargetEncoder

te = TargetEncoder(cols=cat_features)
train_df[cat_features] = te.fit_transform(train_df[cat_features],train_df['ECLO'])
test_df [cat_features] = te.transform(test_df[cat_features])

In [35]:
train_df.head()

,기상상태,노면상태,사고유형,ECLO,구,동,도로구분,도로형태상세,주차장여부,도로여부,기상상태맑음여부,노면상태건조여부,year,month,day,hour,dayofweek,weekend,week,season,week_num,is_holiday,is_dayoff,CCTV_단속구분_1,CCTV_단속구분_2,CCTV_단속구분_4,CCTV_도로노선방향_1,CCTV_도로노선방향_2,CCTV_제한속도_avg,CCTV_설치연도_avg,CCTV_보호구역여부_sum,CCTV_위도_avg,CCTV_경도_avg,보안등_설치개수_sum,보안등_설치연도_avg,보안등_설치형태_건축물,보안등_설치형태_전용주,보안등_설치형태_한전주,보안등_위도_avg,보안등_경도_avg,어린이보호구역_시설종류_어린이집,어린이보호구역_시설종류_유치원,어린이보호구역_시설종류_초등학교,어린이보호구역_시설종류_특수학교,어린이보호구역_시설종류_학원,어린이보호구역_관할경찰서명_강북경찰서,어린이보호구역_관할경찰서명_남부경찰서,어린이보호구역_관할경찰서명_달성경찰서,어린이보호구역_관할경찰서명_대구광역시 중부경찰서,어린이보호구역_관할경찰서명_북부경찰서,어린이보호구역_관할경찰서명_서부경찰서,어린이보호구역_관할경찰서명_수성경찰서,어린이보호구역_CCTV설치여부_N,어린이보호구역_CCTV설치여부_Y,어린이보호구역_CCTV설치대수_sum,어린이보호구역_보호구역도로폭_avg,어린이보호구역_위도_avg,어린이보호구역_경도_avg,주차장_주차장구분_공영,주차장_주차장구분_민영,주차장_주차장유형_노상,주차장_주차장유형_노외,주차장_급지구분_1,주차장_급지구분_2,주차장_급지구분_3,주차장_요금정보_무료,주차장_요금정보_유료,주차장_요금정보_혼합,주차장_주차구획수_avg,주차장_월정기권요금_avg,주차장_평일운영시간_avg,주차장_토요일운영시간_avg,주차장_공휴일운영시간_avg
0,4.712888,4.712878,3.817650,5.0,4.541610,4.282449,4.671841,4.599599,0.0,1.0,1,1,4.842185,4.661716,4.814509,5.071256,4.627926,0,4.702492,4.723309,1,1,1,0.0,1.0,4.0,0.0,1.0,40.0,2013.000000,2.0,35.868237,128.580886,391.0,2007.076046,177.0,30.0,135.0,35.867981,128.579156,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,13.0,26.000000,35.868541,128.581033,0.0,11.0,0.0,11.0,11.0,0.0,0.0,0.0,11.0,0.0,45.454545,0.0,12.636364,12.636364,2.181818
1,4.779150,4.712878,3.817650,3.0,4.618441,4.738938,4.671841,4.599599,0.0,1.0,0,1,4.842185,4.661716,4.814509,5.071256,4.627926,0,4.702492,4.723309,1,1,1,1.0,3.0,8.0,3.0,1.0,52.5,2015.333333,2.0,35.849099,128.540606,932.0,1999.889485,0.0,0.0,0.0,35.849927,128.542160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,35.849513,128.541383,4.0,0.0,1.0,3.0,0.0,1.0,3.0,2.0,2.0,0.0,28.500000,80000.0,18.500000,18.500000,18.500000
2,4.712888,4.712878,3.817650,3.0,4.727300,4.842715,4.671841,4.599599,0.0,1.0,1,1,4.842185,4.661716,4.814509,5.251121,4.627926,0,4.702492,4.723309,1,0,0,2.0,2.0,0.0,3.0,1.0,55.0,2018.250000,0.0,35.834183,128.621395,473.0,2015.334395,14.0,31.0,425.0,35.834061,128.621224,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,4.000000,35.833939,128.621053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,4.712888,4.712878,4.944597,5.0,4.687669,4.208920,4.671841,4.599599,0.0,1.0,1,1,4.842185,4.661716,4.814509,5.407692,4.627926,0,4.702492,4.723309,1,0,0,2.0,8.0,0.0,7.0,3.0,38.0,2018.300000,4.0,35.899975,128.619733,534.0,1990.285714,0.0,0.0,0.0,35.895712,128.619904,3.0,4.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,2.0,9.0,32.0,9.636364,35.897687,128.622803,14.0,0.0,11.0,3.0,0.0,9.0,5.0,13.0,1.0,0.0,26.714286,0.0,23.142857,23.142857,23.142857
4,4.712888,4.712878,4.944597,3.0,4.889534,4.549091,4.671841,4.599599,0.0,1.0,1,1,4.842185,4.661716,4.814509,5.128065,4.627926,0,4.702492,4.723309,1,0,0,2.0,10.0,0.0,9.0,3.0,40.0,2018.250000,7.0,35.883077,128.620268,2057.0,1990.285714,540.0,57.0,1396.0,35.884415,128.623264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,35.883746,128.621766,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,63.000000,80000.0,10.000000,10.000000,10.000000


<br>

## Group scaler

In [36]:
# num_features = train_df.drop(cat_features,axis=1)
# num_features = setdiff(num_features,[CFG.TARGET])

In [37]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = GroupScaler(scaler=MinMaxScaler())
# scaler.fit(
#     data=train_df,
#     group='gungu',
#     num_features=num_features,
# )
# train_df = scaler.transform(train_df)
# test_df  = scaler.transform(test_df)

<br>

## Memory reduction

In [38]:
train_df, _ = reduce_mem_usage(train_df,verbose=False)
test_df , _ = reduce_mem_usage(test_df ,verbose=False)

In [39]:
# train_df.to_parquet('./out/train_data_identity.parquet')
# test_df .to_parquet('./out/test_data_identity.parquet')

<br></br>

# EDA

In [40]:
# d = train_df.copy()

# for i,col in enumerate(num_features):
#     str_i = str(i+1).zfill(len(str(len(num_features))))
#     plt.figure(figsize=(15,7))
#     sns.scatterplot(x=d[col],y=d[CFG.TARGET])
#     plt.grid()
#     plt.title('[{}/{}] {}'.format(str_i,len(num_features),col))
#     plt.show()

# categorical_features = cat_features+dummy_features
# for i,col in enumerate(categorical_features):
#     str_i = str(i+1).zfill(len(str(len(categorical_features))))
#     plt.figure(figsize=(15,7))
#     sns.boxplot(x=d[col],y=d[CFG.TARGET])
#     plt.grid()
#     plt.title('[{}/{}] {}'.format(str_i,len(categorical_features),col))
#     plt.show()

<br></br>

# Modeling

In [41]:
from sklearn.model_selection import KFold
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer

def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    
    y_true = target_transform.inverse_transform(y_true)
    y_pred = target_transform.inverse_transform(y_pred)
    
    if min(y_true)<0: y_true = [max(0,v) for v in y_true]
    if min(y_pred)<0: y_pred = [max(0,v) for v in y_pred]
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

rmsle_scorer = make_scorer(
    name='rmsle',
    score_func=rmsle,
    optimum=0,
    greater_is_better=False,
)

/Users/khj/anaconda3/envs/torch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
unique_info = train_df.nunique()
unique_cols = unique_info[unique_info==1].index.tolist()

train = train_df.drop(unique_cols,axis=1)
test  = test_df .drop(unique_cols,axis=1)
train = TabularDataset(train)
test  = TabularDataset(test)

In [43]:
tune_kwargs = {
    'searcher': 'random',
    'num_trials': 7,
    'scheduler' : 'local'
}

In [44]:
# model = TabularPredictor.load('./AutogluonModels/ag-20231124_074909')

In [45]:
print('start:',datetime.datetime.now())

start: 2023-11-25 16:52:21.095662


In [46]:
%%time

# !rm -rf AutogluonModels
model = TabularPredictor(
    label=CFG.TARGET,
    eval_metric=rmsle_scorer,
    problem_type='regression',
)
model.fit(
    train,
    presets='best_quality',
    hyperparameter_tune_kwargs=tune_kwargs,
    #num_bag_folds=20,
    refit_full=False,
    verbosity=2,
    time_limit=3600*10,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231125_075221/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 36000s
AutoGluon will save models to "AutogluonModels/ag-20231125_075221/"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   1743.59 GB / 2000.20 GB (87.2%)
Train Data Rows:    39609
Train Data Columns: 72
Label Column: ECLO
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3949.28 MB
	Train Data (Original)  Memory Usage: 6.42 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata

[1000]	valid_set's l2: 9.23162	valid_set's rmsle: -0.458491


	-0.4584	 = Validation score   (-rmsle)
	547.98s	 = Training   runtime
	1.45s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1/T5 ... Training model for up to 11014.68s of the 23020.6s of remaining time.
	Fitting 8 child models (S6F1 - S6F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.458	 = Validation score   (-rmsle)
	93.38s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1/T6 ... Training model for up to 10997.53s of the 23003.46s of remaining time.
	Fitting 8 child models (S6F1 - S6F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.4584	 = Validation score   (-rmsle)
	278.6s	 = Training   runtime
	0.74s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1/T7 ... Training model for up to 10948.33s of the 22954.26s of remaining time.
	Fitting 8 child models (S6F1 - S6F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.4581	 = Validation score   (-rmsle)
	124.28s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting 

[1000]	valid_set's l2: 8.77517	valid_set's rmsle: -0.451889


	-0.4559	 = Validation score   (-rmsle)
	195.38s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L2/T7 ... Training model for up to 5058.16s of the 5058.03s of remaining time.
	Fitting 8 child models (S4F1 - S4F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.4555	 = Validation score   (-rmsle)
	106.23s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: LightGBM_BAG_L2/T1 ... Training model for up to 5034.3s of the 5034.17s of remaining time.
	Fitting 8 child models (S4F1 - S4F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.4555	 = Validation score   (-rmsle)
	58.88s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: LightGBM_BAG_L2/T2 ... Training model for up to 5019.7s of the 5019.57s of remaining time.
	Fitting 8 child models (S4F1 - S4F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.4561	 = Validation score   (-rmsle)
	88.01s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: Li

CPU times: user 14h 3min 1s, sys: 8h 53min 57s, total: 22h 56min 58s
Wall time: 9h 44min 50s


In [47]:
##### print(predictor.leaderboard(silent = True))
# predictor.get_model_best()
# model_to_use = predictor.get_model_best()
# model_pred = predictor.predict(test_data, model=model_to_use)

In [48]:
from sklearn.metrics import mean_squared_error

pred = model.predict(train)
pred = [max(0,v) for v in pred]
true = train_df[CFG.TARGET]

pred = target_transform.inverse_transform(pred)
true = target_transform.inverse_transform(true)

rmse_value = mean_squared_error(true,pred)**0.5
rmsle_value = rmsle(true,pred)

print('RMSE: {:.3f}, RMSLE: {:.3f}'.format(rmse_value,rmsle_value))

RMSE: 3.198, RMSLE: 0.447


In [49]:
pred = model.predict(test_df)
pred = [max(0,v) for v in pred]
pred = target_transform.inverse_transform(pred)

In [50]:
# 0.4284723768
submit = pd.read_csv('./data/sample_submission.csv')
submit['ECLO'] = pred
submit.to_csv('./out/19_autogluon_timelimit36000_identity_tune.csv',index=False)
submit.head()

,ID,ECLO
0,ACCIDENT_39609,3.972713
1,ACCIDENT_39610,3.504539
2,ACCIDENT_39611,5.141706
3,ACCIDENT_39612,4.404348
4,ACCIDENT_39613,4.360394
